In [40]:
import pandas as pd
import re

In [41]:
arlette = pd.read_csv("/home/mario/Proyectos_Mario/proyecto_alonso/datos_extraidos/casas_arlette.csv")
arlette = arlette.drop_duplicates()

# Elijo unicamente los inmuebles que son casas
arlette = arlette.query("property_type=='Casa'").reset_index(drop="first")

# Elimino los acentos
arlette["address"] = arlette.address.apply(lambda x: x.lower()\
                                           .replace("á", "a")\
                                            .replace("é", "e")\
                                            .replace("í", "i")\
                                            .replace("ó", "o")\
                                            .replace("ú", "u"))

In [42]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          20 non-null     object 
 1   seller         20 non-null     object 
 2   property_type  20 non-null     object 
 3   address        20 non-null     object 
 4   price          19 non-null     float64
 5   bedrooms       20 non-null     float64
 6   bathrooms      19 non-null     float64
 7   built_area     19 non-null     float64
 8   land_area      19 non-null     float64
 9   description    20 non-null     object 
 10  parking        20 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.8+ KB


In [43]:
arlette = arlette.dropna()
arlette = arlette.reset_index(drop="first")

In [44]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  16 non-null     object 
 3   address        16 non-null     object 
 4   price          16 non-null     float64
 5   bedrooms       16 non-null     float64
 6   bathrooms      16 non-null     float64
 7   built_area     16 non-null     float64
 8   land_area      16 non-null     float64
 9   description    16 non-null     object 
 10  parking        16 non-null     float64
dtypes: float64(6), object(5)
memory usage: 1.5+ KB


### Creando la columna zip_code

In [45]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

In [46]:
# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates()

In [47]:
# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

In [48]:
# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [49]:
# Eliminar palabras que entorpecen la búsqueda del C.P.
arlette["address"] = arlette.address.apply(
    lambda x: x.replace("agencia", "").replace("municipal", "").strip())

In [50]:
# Creo una lista con los datos de zip code para cada propiedad
cp_column = []
for i, d in zip(arlette.index, arlette.address):
    regex = re.compile(fr"(.{{0,20}}{d}.{{0,20}})", re.IGNORECASE)
    location_name = list(filter(lambda x: regex.findall(x), cp_dict.keys()))

    try:
        cp_column.append(cp_dict[location_name[0]])
    except:
        cp_column.append(None)

In [51]:
# Creo la columna zip_code con los valores obtenidos de la iteración
arlette["zip_code"] = cp_column

In [52]:
arlette.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          16 non-null     object 
 1   seller         16 non-null     object 
 2   property_type  16 non-null     object 
 3   address        16 non-null     object 
 4   price          16 non-null     float64
 5   bedrooms       16 non-null     float64
 6   bathrooms      16 non-null     float64
 7   built_area     16 non-null     float64
 8   land_area      16 non-null     float64
 9   description    16 non-null     object 
 10  parking        16 non-null     float64
 11  zip_code       15 non-null     float64
dtypes: float64(7), object(5)
memory usage: 1.6+ KB


In [53]:
arlette = arlette.drop_duplicates()
arlette = arlette.dropna(ignore_index=True)

In [55]:
# arlette.to_csv("datos_arlette.csv", index=False)